# Assignment 6

### Part A

In [199]:
import numpy as np
import pandas as pd

Consider a DataFrame giving information about the price of a stock in 1 year, and the probability of that price occurring.

In [ ]:
data = pd.DataFrame(data = {"Price in 1 Year" : [32, 43, 54],
                            "Probability" : [0.3, 0.4, 0.3]})
                    
data

Write the function ```exp_return```, which calculates the expected return of the stock, given the current stock price ```P```.

In [201]:
def exp_return(data, P):
    final = []
    exp = 0
    input = 0
    c = 0
    
    for index, row in data.iterrows():
        exp = (data.loc[c, "Price in 1 Year"] / P) - 1
        nums = data.loc[c, "Probability"] * exp
        final = final + [nums]
        c+=1
    # exp_return(data, 40) should output 0.07500000000000002
    return sum(final)

In [ ]:
data = pd.DataFrame(data = {"Price in 1 Year" : [32, 43, 54],
                            "Probability" : [0.3, 0.4, 0.3]})
exp_return(data, 40)

Write the function ```volatility```, which calculates the expected return of the stock, given the current stock price ```P```.

In [198]:
def volatility(data, P):
    final = exp_return(data, P)
    answ = []
    exps = 0
    returns = 0
    c = 0
    
    for index, row in data.iterrows():
        returns = (data.loc[c, "Price in 1 Year"] / P) - 1
        exps = data.loc[c, "Probability"] * (returns - final)**2
        answ = answ + [exps]
        c+=1
    return (sum(answ))**(0.5)
    # volatility(data,40) should output 0.21301408404140795

In [ ]:
data = pd.DataFrame(data = {"Price in 1 Year" : [32, 43, 54],
                            "Probability" : [0.3, 0.4, 0.3]})
volatility(data, 40)

Write the function ```interval```, which calculates the 95% confidence interval for the expected return of the data. The corresponding Z score is 1.96. The function should return a list of size 2, with the first element representing the lower bound, and the second element representing the upper bound. 

In [196]:
def interval(data, P):
    vola = volatility(data, P)
    ret = exp_return(data, P)
    
    stand_error = vola / (len(data))**(1/2)
    con_int_low = ret - (1.96 * stand_error)
    con_int_high = ret + (1.96 * stand_error)
    return [con_int_low, con_int_high]
    # interval(data, 40) should output [-0.16604812797447732, 0.3160481279744774]

In [202]:
data = pd.DataFrame(data = {"Price in 1 Year" : [32, 43, 54],
                            "Probability" : [0.3, 0.4, 0.3]})
interval(data, 40)

[-0.16604812797447732, 0.3160481279744774]

Write the function ```exp_return_capm```, which, given the beta of a security ```beta```, the return on the market ```market_return```,  and the risk free interest rate ```rf```, calculates the expected return of the security.

In [ ]:
def exp_return_capm(beta, market_return, rf):
    final = rf + beta*(market_return - rf) 
    return final
    # exp_return_capm(-0.6,0.1, 0.02) should output -0.028

In [ ]:
exp_return_capm(-0.6, .10, .02)

### Part B

Write the function ```clean_data```, which will take a path to a csv file like the one provided. This function should remove all observations with a non number ```RET```, and remove all securities with (strictly) less than 500 observations. Your returned DataFrame should have 4 (non-index) columns, `PERMNO`, `date`, `CUSIP`, and `RET`.

In [48]:
def clean_data(path):
    df = pd.read_csv(path)

    df = df.dropna()
    df = df[df['RET'] != 'C']
    df = df[pd.to_numeric(df['RET'], errors = 'coerce').notnull()]
    
    groups = df.groupby(['CUSIP'], group_keys = False)
    filtered = groups.filter(lambda x: len(x) >= 500).reset_index().drop(['index'], axis = 1)
    
    #df1 = filtered['PERMNO'].value_counts()
    #print(df1)
    # This should yield data with 403316 rows × 4 columns
    return filtered

In [49]:
clean_data("monthlycrsp_mod6.csv")

PERMNO        date     CUSIP        RET
0        10065  1970-01-30  00621210  -0.144375
1        10065  1970-02-27  00621210   0.105882
2        10065  1970-03-31  00621210   0.017857
3        10065  1970-04-30  00621210  -0.078947
4        10065  1970-05-29  00621210  -0.020952
...        ...         ...       ...        ...
403311   84129  2022-08-31  98970110   0.016315
403312   84129  2022-09-30  98970110  -0.075777
403313   84129  2022-10-31  98970110   0.021235
403314   84129  2022-11-30  98970110   0.005583
403315   84129  2022-12-30  98970110  -0.051332

[403316 rows x 4 columns]

Write the function ```avg_return```, which will calculate the average return for each security. This function should take in a DataFrame ```data```, formatted like one outputted by the function ```clean_data```. The results should be returned in a DataFrame formatted like the one supplied.

Additionally, write the function ```security_volatilities```, which does the same thing as ```avg_return```, except it returns the volatilities of the returns of the securities.

In [192]:
def avg_return(data):
    filtered = data
    
    # Creating a dataframe with the CUSIP and number of observations for each security
    df1 = filtered['CUSIP'].value_counts()[filtered.CUSIP.unique()]
    df_val = pd.DataFrame(df1)
    df_obser_cusip = df_val.reset_index()
    df_obser_cusip.columns = ['CUSIP', 'Counts']
    
    df2 = filtered['PERMNO'].value_counts()[filtered.PERMNO.unique()]
    df_val2 = pd.DataFrame(df2)
    df_obser_permno = df_val2.reset_index()
    df_obser_permno.columns = ['PERMNO', 'Counts']
    
    # Creating a DataFrame with numeric Returns 
    new = filtered[['PERMNO', 'RET']].copy()
    new['RET'] = pd.to_numeric(new['RET'], errors='raise')
    
    # List of all valid PERMNOs
    permnos = df_obser_permno['PERMNO'].tolist()
    cusips = df_obser_cusip['CUSIP'].tolist()
    countings = df_obser_cusip['Counts'].tolist()
    
    # Testing
    final = []
    focus = 0
    sums = 0
    nexts = 0

    for i in range(len(permnos)):
        focus = new[new['PERMNO'] == permnos[i]]
        sums = focus['RET'].sum()
        nexts = sums / countings[i]
        final = final + [nexts]
    
    output = pd.DataFrame(data = {
        "CUSIP" : cusips,
        "Average Return" : final
    })
    
    return output
    # This should yield data with 679 rows × 2 columns

In [193]:
avg_return(clean_data("monthlycrsp_mod6.csv"))

CUSIP  Average Return
0    00621210        0.009364
1    43851610        0.012476
2    07373010        0.013279
3    03948310        0.014130
4    00444610        0.009859
..        ...             ...
674  92932Q10        0.015500
675  94331510        0.015024
676  95709010        0.011280
677  98181110        0.016190
678  98970110        0.012356

[679 rows x 2 columns]

In [183]:
def security_volatilities(data):
    filtered = data
    avg_returns = avg_return(data)
    
    # Creating a dataframe with the CUSIP and number of observations for each security
    df1 = filtered['CUSIP'].value_counts()[filtered.CUSIP.unique()]
    df_val = pd.DataFrame(df1)
    df_obser_cusip = df_val.reset_index()
    df_obser_cusip.columns = ['CUSIP', 'Counts']
    
    df2 = filtered['PERMNO'].value_counts()[filtered.PERMNO.unique()]
    df_val2 = pd.DataFrame(df2)
    df_obser_permno = df_val2.reset_index()
    df_obser_permno.columns = ['PERMNO', 'Counts']  
        
    # Creating a DataFrame with numeric Returns 
    new = filtered[['PERMNO', 'RET']].copy()
    new['RET'] = pd.to_numeric(new['RET'], errors='raise')
    
    # List of all valid PERMNOs
    permnos = df_obser_permno['PERMNO'].tolist()
    cusips = df_obser_cusip['CUSIP'].tolist()
    countings = df_obser_cusip['Counts'].tolist()
    returns = avg_returns['Average Return'].tolist()
    
    # Testing
    final = []
    focus = 0
    
    for i in range(len(permnos)):
        sums = 0
        focus = new[new['PERMNO'] == permnos[i]]
        focus_list = focus['RET'].tolist()
        for j in range(len(focus_list)):
            sums = sums + (focus_list[j] - returns[i])**2
        final = final + [(sums / (countings[i] - 1))**(1/2)]
    
    output = pd.DataFrame(data = {
        "CUSIP" : cusips,
        "Volatility" : final
    })
    
    # This should yield data with 679 rows × 2 columns
    return output

In [184]:
security_volatilities(clean_data("monthlycrsp_mod6.csv"))

CUSIP  Volatility
0    00621210    0.049088
1    43851610    0.080908
2    07373010    0.072392
3    03948310    0.084347
4    00444610    0.117598
..        ...         ...
674  92932Q10    0.167238
675  94331510    0.111982
676  95709010    0.073001
677  98181110    0.103132
678  98970110    0.095243

[679 rows x 2 columns]

Write the function ```correlation```, which will return the correlation between average return and volatility. This function takes two DataFrames, ```ret_data``` and ```vol_data```, which are the outputs of ```avg_return``` and ```security_volatilities``` respectively.

In [194]:
def correlation(ret_data, vol_data):
    avg = ret_data['Average Return'].tolist()
    vola = vol_data['Volatility'].tolist()
    
    avg_avg = sum(avg) / len(avg)
    vola_vola = sum(vola) / len(vola)
    
    numerator = 0
    denominator1 = 0
    denominator2 = 0
    for i in range(len(avg)):
        numerator = numerator + ((avg[i] - avg_avg)*(vola[i] - vola_vola))
    
    for i in range(len(avg)):
        denominator1 = denominator1 + (avg[i] - avg_avg)**2
        denominator2 = denominator2 + (vola[i] - vola_vola)**2
        
    final = numerator / (denominator1 * denominator2)**(1/2)
    
    return final
    # correlation(rets, vols) should yield 0.44837090727059614

In [195]:
correlation(avg_return(clean_data("monthlycrsp_mod6.csv")), security_volatilities(clean_data("monthlycrsp_mod6.csv")))

0.44837090727059653